# Sistemas inteligentes. Aplicaciones

## Práctica 4. Autocorrector

En esta práctica vas a implementar un autocorrector. Este sistema es capaz de detectar aquellas palabras que no están escritas correctamente, y reemplaza cada una de ellas por la palabra más probable que esté a una menor distancia de edición (siempre que esta distancia sea menor o igual a 2).

Posteriormente, deberás crear otra función que, dada una frase y su versión corregida, sea capaz de identificar todas las operaciones necesarias de edición para corregirla.

### Creación del diccionario de probabilidades de las palabras del corpus

Vas a comenzar creando el diccionario que necesitas a partir del corpus de entrada. En este caso, el corpus está formado por 21 libros escritos por Pío Baroja (están almacenados en la carpeta libros). Para leer todos ellos, recuerda que a través de la librería ```os``` puedes obtener una lista con todos sus nombres. Los libros están codificados mediante UTF-8, por lo que puedes leerlos con la siguiente instrucción: ```open('nombreArchivo','r',encoding='UTF-8')```.

A partir de ese corpus debes crear un diccionario con todas las palabras que existen en el vocabulario indicando, para cada una de ellas, pa probabilidad de aparición de esa palabra en el corpus. Como solo queremos trabajar con las palabras, debes eliminar del texto todos aquellos símbolos que no sean considerados palabras por una expresión regular (recuerda los elementos ```\w``` y ```\W``` de las expresiones regulares).

In [64]:
import re
import os

files = os.listdir('/Users/juancho/Desktop/Sistemas_Inteligentes/Practica3_Autocorrector/libros')
words = {}
for file in files:
    with open(f'libros/{file}', 'r', encoding='UTF-8') as f:
        for linea in f:
            linea = linea.lower()
            linea = re.sub(r'[^\w\s]','', linea)

            for word in linea.split():
                words[word] = words.get(word,0) + 1

print(words)
print(len(words))
            

{'project': 1923, 'gutenbergs': 8, 'la': 44757, 'lucha': 145, 'por': 11484, 'vida': 1345, 'mala': 338, 'hierba': 89, 'by': 609, 'pío': 104, 'baroja': 124, 'this': 1084, 'ebook': 255, 'is': 517, 'for': 572, 'the': 4083, 'use': 242, 'of': 2687, 'anyone': 109, 'anywhere': 43, 'at': 330, 'no': 15636, 'cost': 65, 'and': 1562, 'with': 1047, 'almost': 43, 'restrictions': 43, 'whatsoever': 43, 'you': 1581, 'may': 330, 'copy': 263, 'it': 327, 'give': 87, 'away': 43, 'or': 1713, 'reuse': 43, 'under': 131, 'terms': 461, 'gutenberg': 659, 'license': 329, 'included': 65, 'online': 124, 'wwwgutenbergorglicense': 24, 'title': 21, 'author': 21, 'release': 21, 'date': 67, 'june': 2, '25': 18, '2013': 2, '43033': 1, 'language': 21, 'spanish': 21, 'start': 65, 'produced': 88, 'chuck': 4, 'greif': 4, 'distributed': 125, 'proofreading': 37, 'canada': 4, 'team': 37, 'httpwwwpgdpcanadanet': 4, 'book': 10, 'was': 71, 'from': 365, 'scanned': 2, 'images': 35, 'public': 112, 'domain': 90, 'material': 16, 'google

### Funciones para aplicar una operación de edición
Ahora debes crear 4 funciones, cada una relacionada con una posible operación de edición: insertar, borrar, reemplazar e intercambiar. Cada una de estas funciones debe recibir una palabra y devolver una lista con todos los strings que se puedan obtener al aplicar esa operación de edición sobre la palabra recibida.

In [65]:
def insert(word: str) -> set:
    alphabet = 'abcdefghijklmnñopqrstuvwxyz'
    words_created = set()
    for i in range(len(word) + 1):
        for char in alphabet:
            new_word = word[:i] + char + word[i:]
            words_created.add(new_word)
    
    return words_created



In [66]:
def delete(word:str) -> set:
    words_created = set()
    for i in range(len(word)):
        new_word = word[:i] + word[i+1:]
        words_created.add(new_word)

    return words_created
        
word = 'hola'
delete(word)

{'hla', 'hoa', 'hol', 'ola'}

In [67]:
def replace(word:str) ->set:
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    words_created = set()
    for i in range(len(word)): #hola
        for char in alphabet:
            new_word = word[:i] + char + word[i+1:]
            words_created.add(new_word)

    return words_created



In [68]:
def exchange(word:str)->set:
    words_created = set()
    for i in range(len(word) -1):
        new_word = word[:i] + word[i+1] + word[i] + word[i+2:]
        words_created.add(new_word)

    return words_created

word = 'hola'
exchange(word)

    #hola -> ohla, hloa, hoal

{'hloa', 'hoal', 'ohla'}

### Función una_edicion
Utilizando las 4 funciones anteriores, crea una nueva función que, recibiendo una palabra, devuelva todos lo strings que se encuentran a distancia 1 de edición de dicha palabra.

Además de la palabra, esta función debe recibir como argumento el parámetro *intercambiar*, que por defecto vale *False*. Si este parámetro es verdadero, utilizamos las 4 operaciones de edición (insertar, borrar, reemplazar e intercambiar). Si este parámetro es falso, entonces solo utilizamos como operaciones de edición insertar, borrar y reemplazar.

Ten en cuenta que al aplicar dos operaciones de edición por separado, podríamos obtener un mismo string. Debes asegurarte de que en los strings que devuelve tu función no hay ninguno repetido.

In [69]:
def una_edicion(word:str, intercambiar: bool=False) ->set:
    set1 = insert(word)
    set2 = delete(word)
    set3 = replace(word)
    if not intercambiar:
        words_one_edition = set1.union(set2, set3)
    else:
        set4 = exchange(word)
        words_one_edition = set1.union(set2, set3, set4)

    return words_one_edition


### Función dos_ediciones

Crea una función que recibe una palabra y devuelve todos los strings que se encuentran a 2 unidades de distancia de edición de esa palabra. Al igual que en el caso anterior, utiliza un pa´rametro intercambiar para permitir usar o no este tipo de operación de edición.

Vuelve a tener en cuenta que no debes devolver strings repetidos, aunque hayan sido creados aplicando diferentes operaciones de edición.

In [70]:
def dos_ediciones(word:str,intercambiar:bool =False) ->set:
    palabras_ed1 = una_edicion(word, intercambiar)
    palabras_ed2 = set()
    for palabra in palabras_ed1:
        palabras_ed2 = palabras_ed2.union(una_edicion(palabra, intercambiar))
    return palabras_ed2

### Función corregir

Utilizando las funciones anteriores, crea una nueva función que recibe una palabra y devuelve la o las correcciones sugeridas. Esta función recibe como argumentos la palabra a corregir, el diccionario de probabilidades de cada palabra del corpus, un valor entero (n) que indica el número máximo de correcciones sugeridas a devolver y un valor booleano que indica si queremos utilizar la operación de edición intercambiar o no.

Según la palabra recibida, esta función debe devolver:
* si la palabra existe en el diccionario, la devuelve tal cual
* si no
    * si existen palabras reales a distancia una, devuelve las n más probables, en orden de mayor a menor
    * si no
           * si existen palabras reales a distancia dos, devuelve las n más probables, en orden de mayor a menor
           * si no, devuelve la palabra original

El parámetro del número máximo de palabras a devolver se utiliza solo entre palabras que estén a la misma distancia de edición. Por ejemplo, si n=10 y existen 2 palabras a distancia de edición 1 y 20 palabras a distancia de edición 2, la función solo devolverá las dos palabras a distancia 1. Si, en otro caso, n=10 y existen 15 palabras a distancia de edición 1 y 30 palabras a distancia de edición 2, la función devolverá las 10 palabras más probables que están a distancia de edición 1.

In [ ]:
def correct(word: str, corpus: dict, n: int, intercambiar: bool = False) -> list:
    if word in corpus:
        return [word]
    
    palabras_ed1 = una_edicion(word, intercambiar)
    validated_words_ed1 = {p for p in palabras_ed1 if p in corpus}
    
    if validated_words_ed1:
        sorted_words = sorted(validated_words_ed1, key=lambda x: corpus[x], reverse=True)
        return sorted_words[:n]
    
    palabras_ed2 = dos_ediciones(word, intercambiar)
    validated_words_ed2 = {p for p in palabras_ed2 if p in corpus}
    if validated_words_ed2:
        sorted_words = sorted(validated_words_ed2, key=lambda x: corpus[x], reverse=True)
        return sorted_words[:n]
    
    return [word]


['horas', 'olas', 'hojas']

### Ejecución

Ejecuta las funciones anteriores para, leída una palabra del usuario, devuelva las (máximo 5) palabras reales más probables a menor distancia de edición.

In [74]:
correct('holaas',words,5)

['horas', 'olas', 'hojas', 'hola', 'solas']

### Corrección de una frase

En este caso vamos a trabajar con una frase completa. Para cada una de las palabras que no sean correctas, debes cambiarlas por la palabra que esté a menor distancia de edición que sea más probable. Una vez que tienes la frase corregida, llama a otra función que calcule el número mínimo de operaciones de edición necesarias para pasar de la frase original a la corregida.

En este caso, fijamos que no se puede utilizar la operación de intercambiar y los costes de las otras tres operaciones los pasamos como parámetro a la función que calcula la distancia de edición.

Para calcular la distancia de edición mínima, utiliza el algoritmo de programación dinámica, y devuelve tanto la distancia mínima como la matriz calculada.

### Todas las posibles ediciones
Para finalizar, escribe una función que recibe dos palabras (la original y la corregida) y la matriz de distancias mínimas calculadas mediante el algoritmo de programación dinámica. Esta función debe devolver la lista de ediciones que se debe hacer sobre la palabra original para conseguir la palabra corregida.

Como puede haber varios conjuntos de ediciones que obtengan la palabra corregida con el mísmo número de ediciones, debes devolver todos ellos.